In [8]:
import os
import random
os.chdir("..")

In [9]:
import os
import pandas as pd
import random
import time

# Settings
ADP_FOLDER = "adp"
SEASONAL_STATS_FOLDER = "seasonalstats"
DEFENSIVE_STATS_FOLDER = "defensivestats"
RESULTS_FOLDER = "results"
NUMBER_OF_TRIALS = 3
NUM_MANAGERS = 12
NUM_ROUNDS = 16

# Position limits and requirements
POSITION_LIMITS = {"QB": 4, "RB": 8, "WR": 8, "TE": 3, "K": 3, "DST": 3}
REQUIRED_POSITIONS = {"QB": 1, "K": 1, "DST": 1, "RB": 2, "WR": 2, "TE": 1}

# Weighted probabilities for player selection
ROUND_1_3_WEIGHTS = [0.64, 0.20, 0.10, 0.05, 0.01]  # Rounds 1–3
ROUND_4_16_WEIGHTS = [0.50, 0.10, 0.10, 0.10, 0.10, 0.10]  # Rounds 4–16


# Utility: Load files
def load_file(folder, filename):
    file_path = os.path.join(folder, filename)
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    raise FileNotFoundError(f"File {filename} not found in folder {folder}")


# Load ADP file
def load_adp_file():
    year = random.choice([2018, 2019, 2020, 2021, 2022, 2023])
    file_name = f"{year}ADP.csv"
    adp_df = load_file(ADP_FOLDER, file_name)
    adp_df['year'] = year  
    return adp_df


# Load stats
def load_seasonal_stats(year):
    return load_file(SEASONAL_STATS_FOLDER, f"player_stats_{year}.csv")


def load_defensive_stats(year):
    return load_file(DEFENSIVE_STATS_FOLDER, f"seasonal_defensive_stats_{year}.csv")


# Merge stats into ADP
def merge_stats(adp_df, seasonal_stats_df, defensive_stats_df):
    adp_df = adp_df.merge(
        seasonal_stats_df[["player_id", "fppr"]], on="player_id", how="left"
    )
    defensive_stats_df = defensive_stats_df.rename(columns={"pa_team": "player_id", "fpts": "def_fpts"})
    adp_df = adp_df.merge(
        defensive_stats_df[["player_id", "def_fpts"]], on="player_id", how="left"
    )
    adp_df["fpts"] = adp_df.apply(
        lambda row: row["def_fpts"] if row["POSITION"] == "DST" else row["fppr"], axis=1
    )
    return adp_df


# Select player with weighted probabilities
def select_player_with_weights(players, weights):
    """
    Randomly select a player based on weighted probabilities.
    """
    return random.choices(players.to_dict("records"), weights=weights[:len(players)], k=1)[0]



NameError: name 'adp_folder' is not defined